In [2]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 22})

import numpy as np
import itertools
import glob

import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
from parse_cube_files import CUBE
import alchemy_tools as at


# Import data into cube objects

In [5]:
paths = glob.glob('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212/cube-files/*.cube')
paths.sort()

cube_obj = []
for p in paths:
    cube_obj.append(CUBE(p))

In [6]:
# uniform electron gas
path_ueg = '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/ueg/ve_00.cube'
cube_obj.insert(0, CUBE(path_ueg))

# Example 1D

In [203]:
ve_ex = cube_obj[len(cube_obj)-2]

fig_m, ax_m = plt.subplots(1,2)
x = np.linspace(0, 20, 175)
pr = ve_ex.project((0,2))
# ax_m.plot(x, pr, label='full')

fft = np.fft.fft(pr)
fft_scaled = fft.copy()
fft_scaled[0] = fft_scaled[0]
fft_scaled[1] = fft_scaled[1]*1.5
fft_scaled[2] = fft_scaled[2]*0.75

# fft_scaled[5] = 0.079#fft_scaled[5]
# fft_scaled[6] = 0.0721#fft_scaled[6]
# fft_scaled[7] = 0.070#fft_scaled[7]

ifft= np.fft.ifft(fft_scaled)

ax_m[0].plot(np.arange(len(fft))[0:25],np.abs(fft)[0:25], '-o', label='original')
ax_m[0].plot(np.arange(len(fft))[0:25],np.abs(fft_scaled)[0:25], '-o', label='rescaled')

ax_m[1].plot(x, pr, label="original")
ax_m[1].plot(x, ifft.real, label="rescaled")




ax_m[0].legend()
ax_m[1].legend()

In [204]:
ve_ex = cube_obj[1]
fig_i, ax_i = plt.subplots()
x = np.linspace(0, 20, 175)
pr = ve_ex.project((0,2))
ax_i.plot(x, pr, label='full')
for idx in range(0,3):
    pr = ve_ex.project((0,1))
    fft = np.fft.fft(pr)
    fft[idx] = 0.0
    ifft= np.fft.ifft(fft)
    ax_i.plot(x, ifft.real, label="without freq {}".format(idx))
ax_i.set_title('projection (0,1)')
ax_i.legend()

In [215]:
# density for different lambda-values
# and absolute of fourier coefficients for the first 25 frequencies at every density

fig_m, ax_m = plt.subplots(1,2)
x = np.linspace(0, 20, 175)
fft_freqs = np.empty((len(cube_obj), 25), dtype='complex128')
n = 5
lam_val = np.array([0, 4, 6, 8, 10, 12, 14, 15, 16, 18, 20, 22, 23, 24, 26, 28, 29, 30, 32, 34, 36, 38])/38
for i in range(len(cube_obj)):
    ve_ex = cube_obj[i]
    pr = ve_ex.project((1,2))

    fft = np.fft.fft(pr)
    fft_scaled = fft.copy()

    ifft= np.fft.ifft(fft_scaled)

    ax_m[0].plot(np.arange(len(fft))[0:25],np.abs(fft)[0:25], '-o')

    ax_m[1].plot(x, pr)
    
    fft_freqs[i] = fft[0:25]


ax_m[0].set_xlabel(r'frequency $\nu$')
ax_m[0].set_ylabel(r'$|c_{\nu}|$')
ax_m[0].set_title(r'Absolute of fourier coefficents for $\rho(\lambda)$')

ax_m[1].set_xlabel(r'cell coordinate $x_{1,2}$')
ax_m[1].set_ylabel(r'$\rho(x_{1,2})$')
ax_m[1].set_title(r'1D projection of density for different $\lambda$')


# only densities
# fig_m, ax_m = plt.subplots(1,1)
# x = np.linspace(0, 20, 175)
# fft_freqs = np.empty((len(cube_obj), 25), dtype='complex128')
# n = 5
# lam_val = np.array([0, 4, 6, 8, 10, 12, 14, 15, 16, 18, 20, 22, 23, 24, 26, 28, 29, 30, 32, 34, 36, 38])/38
# for i in range(len(cube_obj)):
#     ve_ex = cube_obj[i]
#     pr = ve_ex.project((1,2))
#     ax_m.plot(x, pr)

# ax_m.set_xlabel(r'cell coordinate $x_{1,2}$')
# ax_m.set_ylabel(r'$\rho(x_{1,2})$')
# ax_m.set_title(r'1D projection of density for different $\lambda$')

Text(0.5, 1.0, '1D projection of density for different $\\lambda$')

In [233]:
# fourier coefficients real, imag, absolute at every lambda value

fig_coeff, ax_coeff = plt.subplots(1,3)
plt.rcParams.update({'font.size': 18})
fig_coeff.suptitle(r'Fourier coefficients of the 5 lowest frequencies as a function of $\lambda$' + '\n' + 'for the 1D-projected densities of dsgdb9nsd_001212')
plt.rcParams.update({'font.size': 22})
for i in range(0,5):#(len(cube_obj)):
    ax_coeff[0].plot(lam_val, fft_freqs.real[:,i], '-o', label = r'$\nu = ${}'.format(i))
    ax_coeff[1].plot(lam_val, fft_freqs.imag[:,i], '-o', label = r'$\nu = ${}'.format(i))
    ax_coeff[2].plot(lam_val, np.abs(fft_freqs[:,i]), '-o', label = r'$\nu = ${}'.format(i))

ax_coeff[0].set_xlabel(r'$\lambda$')
ax_coeff[0].set_ylabel(r'$c_{\nu, real}(\lambda)$')
ax_coeff[0].legend()

ax_coeff[1].set_xlabel(r'$\lambda$')
ax_coeff[1].set_ylabel(r'$c_{\nu, im}(\lambda)$')
ax_coeff[1].legend()

ax_coeff[2].set_xlabel(r'$\lambda$')
ax_coeff[2].set_ylabel(r'$|c_{\nu}(\lambda)|$')
# ax_coeff[2].legend()

Text(0, 0.5, '$|c_{\\nu}(\\lambda)|$')